Calculate In-Degree and Out-Degree of Nodes for DiGraphs and Degree of Nodes in Graphs

In [ ]:
import networkx as nx

def degree_calculator(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    if nx.is_directed(G):
        print("\n--- Calculate In-Degree And Out-Degree ---\n")
        choice = input("Do you want the degree of a specific node or all nodes (specific/all)? ").lower()
        if choice == "all":
            print("\n--- In-degree and Out-degree of all nodes ---")
            for node in G.nodes():
                print(f"Node {node}: In_Degree = {G.in_degree(node)}, Out_Degree = {G.out_degree(node)}")
        elif choice == "specific":
            node = input("Enter the desired node name: ")
            try:
                node = int(node)
            except ValueError:
                pass
            if G.has_node(node):
                print(f"Node {node}: In_Degree = {G.in_degree(node)}, Out_Degree = {G.out_degree(node)}")
            else:
                print(f"Node {node} doesn't exist in the graph.")
        else:
            print("Invalid choice.")
            return

    else:
        print("\n--- Calculate Degree ---\n")
        choice = input("Do you want the degree of a specific node or all nodes (specific/all)? ").lower()
        if choice == "all":
            print("\n--- Degree of all nodes ---")
            for node in G.nodes():
                print(f"Node {node}: Degree = {G.degree(node)}")
        elif choice == "specific":
            node = input("Enter the desired node name: ")
            try:
                node = int(node)
            except ValueError:
                pass
            if G.has_node(node):
                print(f"Node {node}: Degree = {G.degree(node)}")
            else:
                print(f"Node {node} doesn't exist in the graph.")
        else:
            print("Invalid choice.")
            return


Find Strongly Connected Components for DiGraphs and Connected Components for Graphs

In [ ]:
def find_connected_components(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    if nx.is_directed(G):
        print("\n--- Find Strongly Connected Components ---")
        strong_components = list(nx.strongly_connected_components(G))
        for i, component in enumerate(strong_components):
            print(f"Component {i+1}: {component}")

    else:
        print("\n--- Find Connected Components ---")
        connected_components = list(nx.connected_components(G))
        for i, component in enumerate(connected_components):
            print(f"Component {i+1}: {component}")

Find Shortest Path Between Nodes Using Dijkstra's Algorithm

In [ ]:
def find_shortest_path(G, source, target):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    print("\n--- Find shortest path ---")
    try:
        path = nx.dijkstra_path(G, source, target, weight='weight')
        distance = nx.dijkstra_path_length(G, source, target)
        print(f"\nThe shortest path between {source} and {target} is {path}")
        print(f"Path length: {distance}")
    except nx.NetworkXNoPath:
        print(f"No path exists between {source} and {target}.")


Analyze Nodes Centrality

In [ ]:
def analyze_centrality(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    print("\n--- Centrality Analysis ---")

    print("\n1. Degree Centrality (higher is more important):")
    degree_centrality = nx.degree_centrality(G)
    for node, centrality in sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")
    if nx.is_directed(G):
        print("\n   → In-Degree Centrality")
        in_degree_centrality = nx.in_degree_centrality(G)
        for node, centrality in sorted(in_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

        print("\n   → Out-Degree Centrality")
        out_degree_centrality = nx.out_degree_centrality(G)
        for node, centrality in sorted(out_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

    print("\n2. Betweenness Centrality (nodes on shortest paths):")
    betweenness_centrality = nx.betweenness_centrality(G)
    for node, centrality in sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n3. Closeness Centrality (average distance to all others):")
    closeness_centrality = nx.closeness_centrality(G)
    for node, centrality in sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n4. Eigenvector Centrality (nodes on shortest paths):")